In [14]:
#import libraries
#import pyreadr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()
import pickle
pd.set_option('display.max_rows', 1000)
from scipy.optimize import lsq_linear
import sklearn
#from mlutils import dataset, connector
pd.set_option('display.max_columns',1000)

import tensorflow as tf

In [15]:
import configparser
configParser = configparser.RawConfigParser()   
configFilePath = './config.properties'
configParser.read(configFilePath)

['./config.properties']

In [16]:
# dept_to_run=configParser.get('modelling', 'dept_to_run')
# dept_to_run=[int(each_dept) for each_dept in dept_to_run.split(',')]

#input path
item_matl_path = (configParser.get('data_paths', 'prj_dir_path') \
                              + configParser.get('data_paths', 'item_matl_model_data_csv_path'))



single_estimates_path = (configParser.get('data_paths', 'prj_dir_path') \
                              + configParser.get('estimates', 'single_estimates_path'))
single_estimates_coo_path = (configParser.get('data_paths', 'prj_dir_path') \
                              + configParser.get('estimates', 'single_estimates_coo_path'))
ihs_estimates_path=configParser.get('estimates', 'ihs_estimates')
ihs_estimates_coo_path=configParser.get('estimates', 'ihs_estimates_coo')

#output path
bootstrap_estimates_path = (configParser.get('data_paths', 'prj_dir_path') \
                              + configParser.get('estimates', 'bootstrap_estimates'))


In [17]:
%%time
#reading item material table
item_matl=pd.read_csv(item_matl_path,low_memory=False)
print(item_matl.shape)

(22626, 86)
CPU times: user 305 ms, sys: 14.5 ms, total: 319 ms
Wall time: 323 ms


## Importing calculated Single Item estimates

In [18]:
with open(single_estimates_coo_path, 'rb') as handle:
    single_item_coo_estimates = pickle.load(handle)
with open(single_estimates_path, 'rb') as handle:
    single_item_estimates = pickle.load(handle)

## Importing calculated IHS cost estimates

In [19]:

ihs=pd.read_pickle(ihs_estimates_path)
ihs_coo=pd.read_pickle(ihs_estimates_coo_path)

In [20]:
ihs_estimates_ll=ihs.set_index('matl_desc_cln')['auc_lb_ll'].to_dict()
ihs_estimates_ul=ihs.set_index('matl_desc_cln')['auc_lb_ul'].to_dict()
ihs_coo_estimates_ll=ihs_coo.set_index('matl_desc_cln_coo')['auc_lb_ll'].to_dict()
ihs_coo_estimates_ul=ihs_coo.set_index('matl_desc_cln_coo')['auc_lb_ul'].to_dict()

In [21]:
def return_misc_cost_group_per_dept(item_matl,dept):
    return item_matl.loc[item_matl['acctg_dept_nbr']==dept,'misc_cost_group'].unique()

## Running least squares Model

In [22]:
def createModellingDataset(item_matl, dept_no, target, min_matl_freq = 5):
    '''
    --method performs
    1.filtering items only if all materials have greater than 5 occurances across products
    2. matl cost is constant by dept and coo combination hence matl_desc and coo is merged
    3. misc cost is constant by creating one hot encoding for acctg_dept_nbr_dept_category_nbr_dept_subcatg_nbr_fineline_nbr_vendor_nbr
    --inputs:
    item_matl: input the item_matl datset
    dept_no: input the dept no for which modelling datset needs to be created
    
    --output:
    modelling df where columns is matl_desc_coo combination, index is item_fam_id,values are matl_perc
    
    '''
    dept=item_matl[(item_matl.acctg_dept_nbr==dept_no)&(~item_matl[target].isnull())].copy()
    
    #filtering items only if  materials have greater than 5 occurances across items in same dept
    ans=dept.groupby('matl_desc_cln')['item_nbr'].agg('count')
    list_of_items=ans[ans>=min_matl_freq].index
    dept['no_of_matl']=dept.groupby('item_nbr')['item_nbr'].transform('count')
    dept['flag_freq_matl']=dept['matl_desc_cln'].isin(list_of_items)

    dept['sum_flag_freq_matl']=dept.groupby('item_nbr')['flag_freq_matl'].transform('sum')
    dept=dept[dept['sum_flag_freq_matl']==dept['no_of_matl']]
    #dept=dept.reset_index(drop=True)
    
    
#     dept['misc_cost_group']=dept['acctg_dept_nbr'].astype(int).astype('str')+'_'+\
#         dept['dept_category_nbr'].astype(int).astype('str')+'_'+\
#         dept['dept_subcatg_nbr'].astype(int).astype('str')+'_'+\
#         dept['fineline_nbr'].astype(int).astype('str')+'_'+\
#         dept['vendor_nbr'].astype(int).astype('str')
    #dept['matl_desc_cln_coo']=dept['matl_desc_cln'].astype('str')+'_'+dept['coo'].astype('str')
    
    df=pd.DataFrame(index=dept.mds_fam_id.unique(),columns=dept.matl_desc_cln.unique())
    
    for index,row in dept.iterrows():
        df.loc[row['mds_fam_id'],row['matl_desc_cln']]=row['matl_pct']
        df.loc[row['mds_fam_id'],target]=row[target]
        df.loc[row['mds_fam_id'],'misc_cost_group']=row['misc_cost_group']

    df=df.fillna(0)
    

    #one hot encoding
    df_one_hot_encoded=pd.get_dummies(df['misc_cost_group'])
    df=df.drop('misc_cost_group',axis=1)

    #concat final df
    df_concat=pd.concat([df,df_one_hot_encoded],axis=1)
    
    #dropping duplicates
    df_concat=df_concat.drop_duplicates(keep='first')   
    
    #df=df.loc[df_concat.index,:]
    #df_one_hot_encoded=df_one_hot_encoded.loc[df_concat.index,:]

    return df_concat #,df,df_one_hot_encoded

In [23]:
def lowerAndUpperBounds(df,target,
                        ihs_coo_estimates_ul,ihs_coo_estimates_ll,
                        ihs_estimates_ul,ihs_estimates_ll,
                        single_item_coo_estimates,single_item_estimates,misc_cost_groups,
                        defaultub=np.inf,defaultlb=0.01):
    ub=[]
    lb=[]
    for each in df.columns:
        material=each.split('_')[0]

        if each in target:
            continue
        elif each in ihs_coo_estimates_ul.keys():
            #print(each)
            lower=ihs_coo_estimates_ll[each]
            upper=ihs_coo_estimates_ul[each]
        elif material in ihs_estimates_ul.keys():
            #print(material)
            lower=ihs_estimates_ll[material]
            upper=ihs_estimates_ul[material]
        elif each in single_item_coo_estimates.keys():
            #print(each)
            lower=defaultlb
            upper=single_item_coo_estimates[each]
        elif material in single_item_estimates.keys():
            #print(material)
            lower=defaultlb
            upper=single_item_estimates[material]
        elif each in misc_cost_groups:
            #print(each)
            lower=0.100
            upper=0.900
        else:
            #print(each)
            lower=defaultlb
            upper=defaultub   

        if lower==upper:
            lower=upper*0.70

        if ~np.isinf(upper):
            if (upper-lower)/upper < 0.30:
                lower=upper*0.70

        ub.append(upper)
        lb.append(lower)


    ub=np.array(ub)
    lb=np.array(lb)
    
    return lb,ub

In [24]:
def createDependentIndependentVars(df,misc_cost_groups):
    #div all df with target col
    #df=df.div(df['fc_lb'],axis=0)

    #creating independent and dependent variables
    X=df.drop('fc_lb',axis=1)

    #take backup of columns names in independent variables
    cols=X.columns

    #creating 2 independent matrices for materials and fine line vendor combination
    X1=X.loc[:,~X.columns.isin(misc_cost_groups)]
    X2=X.loc[:,X.columns.isin(misc_cost_groups)]
    
    X1 = X1.div(df['fc_lb'],axis=0)
    Y = df['fc_lb']/df['fc_lb']
    
    
    return X1,X2,Y,cols

def reshapeArrays(X1,X2,Y,lb,ub,init):
    
    # m is no of rows , n is no of columns

    #converting weight to (n,1) shape
    lb=np.array(lb).reshape(lb.shape[0],1)
    ub=np.array(ub).reshape(ub.shape[0],1)
    init=np.array(init).reshape(init.shape[0],1)

    #converting X to (n,m) shape
    X1=np.array(X1).T
    X2=np.array(X2).T

    #matrix multiplication of W.T,X --> (1,n)*(n,m) -->(1,m)

    #convering Y to (1,m) shape

    Y=np.array(Y).reshape(1,Y.shape[0])
    
    return X1,X2,Y,lb,ub,init

def coeffsInitilization(lb,ub):
    #initilization point for the optimizer
    init=(lb+ub)/2
#     init[np.isinf(init)]=lb[np.isinf(init)]+1e-8
    init[np.isinf(init)]=lb[np.isinf(init)] + 1
    
    return init


def rand_coeffsInitilization(lb, ub, coeffsInitPoint = []):
    #initilization point for the optimizer
    init=(lb+ub)/2
    
    for k in range(len(lb)):
        if np.isinf(init[k]):
            init[k]=lb[k] + 1
        else:
            init[k] = np.random.uniform(low=lb[k], high=ub[k], size = 1)

    if len(coeffsInitPoint)==len(lb):
        init = (init + coeffsInitPoint)/2
    
    return init


In [31]:
#optmizer

def optimize(X1, X2, Y, coeffsInitPoint, lb, ub, epochs=10**6, verbose=True, method='ADAM'
             , loss_lambda = 1.0,under_est_pct=0.7):
    
    #reshape arrays
    
    X1,X2,Y,lb,ub,coeffsInitPoint=reshapeArrays(X1,X2,Y,lb,ub,coeffsInitPoint)
    
    item_cnt = Y.shape[1]
    
    #create tensorflow variables
    def constr(a, b):
        return lambda x: tf.clip_by_value(x, a, b)


    Y = tf.Variable(Y, dtype=tf.float32, name='Y')
    coeffs = tf.Variable(coeffsInitPoint, trainable=True, dtype=tf.float32, name='coeffs', constraint=constr(lb,ub))
    X1 = tf.Variable(X1, dtype=tf.float32, name='X1')
    X2 = tf.Variable(X2, dtype=tf.float32, name='X2')

    
    #create loss function with non linear equation
    
    # y=(matl_pct1*matl_cost1+matl_pct2*matl_cost2+...)/(1-fineline_vendor_margin)
    def model(X1,X2,coeffs):
        raw_mat_cost=tf.matmul(tf.transpose(coeffs[:X1.shape[0]]),X1)
        fvp=tf.matmul(tf.transpose(coeffs[X1.shape[0]:]),X2)
#         pred=raw_mat_cost/(1-fvp)
        pred=raw_mat_cost+fvp
        return pred

    def loss_fn():
        Y_PRED=model(X1,X2, coeffs)

#         loss = tf.reduce_mean(tf.math.square(Y_PRED - Y)+ loss_lambda*(Y_PRED - Y)) 

#         loss = tf.reduce_mean(tf.math.square(Y_PRED - Y) + (item_cnt/1000)*(Y_PRED - Y)) # resulted in 100% underestimation
#         loss = tf.reduce_mean(tf.math.square(Y_PRED - Y) + 1*(0.80 - 1*(Y_PRED < Y))) # not differentiable
#         loss = tf.reduce_mean(tf.math.square(Y_PRED - Y)) + 1*(0.80 - tf.reduce_mean(Y_PRED < 1)) # not differentiable
        
        loss = tf.reduce_mean(tf.math.square(Y_PRED - Y)) + loss_lambda*tf.abs(under_est_pct - tf.reduce_mean(tf.where(Y_PRED < Y, 1.0, 0.0)))


        return loss


    #optimizer
    
    if method == 'SGD':
        opt = tf.keras.optimizers.SGD(learning_rate=0.01)
 
    if method == 'ADAM':
        opt = tf.keras.optimizers.Adam(learning_rate=0.01,
                                       beta_1=0.9,
                                       beta_2=0.999,
                                       epsilon=1e-07,
                                       amsgrad=False)

    obj_vals = []
    weights = []

    for epoch in range(epochs):
        
        #printing loss_function value after every 10000 epochs
        if ((verbose) & (epoch % 10000 == 0)):
            print(f'step: {epoch}, obj = {loss_fn().numpy():.2f}')
            
        obj_vals.append(loss_fn().numpy())
        weights.append(coeffs.numpy())
        
        
        # early stopping
        if epoch>100:
            
            cur_loss_avg =  np.mean(obj_vals[(epoch-5):(epoch)])
            prev_loss_avg = np.mean(obj_vals[(epoch-10):(epoch-6)]) 
            
            if np.abs(cur_loss_avg - prev_loss_avg) < .00001 :
                break
        
        opt.minimize(loss_fn, var_list=coeffs)
    
    if verbose:
        print(f'step: {epoch}, obj = {loss_fn().numpy():.2f}')
    
    return obj_vals, weights

In [32]:
def simple_resample(n): 
    return(np.random.randint(low = 0, high = n, size = n))

In [33]:
def runBootStrapGDModel(df, misc_cost_groups, lb, ub, 
                        epochs=10**6, verbose=True, method='ADAM', noIterations=0, runBootStrap=False,
                        loss_lambda = 1.0):
    '''
    This methods runs bootstrap least square model with bounds
    
    input:
    df: modelling dataframe
    target_col: dependent variable
    noIterations: no of iterations to run bootstrap
    
    output:
    estimates dataframe
    estimates percentile dataframe
    metrics dataframe
    
    '''
    
    coeffsInitPoint_avg = coeffsInitilization(lb, ub)
    X1,X2,Y,cols=createDependentIndependentVars(df,misc_cost_groups)
    
    coeffs_list=[]
    obj_vals, weights = optimize(X1, X2, Y, 
                                 coeffsInitPoint_avg, 
                                 lb, ub, 
                                 epochs=epochs, 
                                 verbose=verbose,
                                 method=method,
                                 loss_lambda = loss_lambda)
    
    coeffs_list.append(weights[obj_vals.index(min(obj_vals))].reshape(-1))
    
    coeffsInitPoint_all = coeffs_list[0]
    
    #bootstrap regression

    if runBootStrap==True:
        
        print('********** bootstrap regression started **********')

        X1=X1.reset_index(drop=True)
        X2=X2.reset_index(drop=True)

        for i in range(0,noIterations):
            if i%10==0:
                print('***** bootstrap iteration ' + str(i) + ' *****')
            
            np.random.seed(i)
            coeffsInitPoint = rand_coeffsInitilization(lb, ub, coeffsInitPoint_all)
#             coeffsInitPoint = coeffsInitPoint_all

            indices=simple_resample(X1.shape[0])
            X_bootstrap1=X1.loc[indices,:]
            X_bootstrap2=X2.loc[indices,:]
            y_bootstrap=Y.values[indices]
            
            obj_vals, weights = optimize(X_bootstrap1,
                                         X_bootstrap2,
                                         y_bootstrap,
                                         coeffsInitPoint,
                                         lb,
                                         ub,
                                         epochs=epochs,
                                         verbose=(i%10==0), 
                                         method=method,
                                        loss_lambda = loss_lambda)
            
            
            coeffs_list.append(weights[obj_vals.index(min(obj_vals))].reshape(-1))    

        print('********** bootstrap regression done **********')     
    
    estimates_df=pd.DataFrame(np.array(coeffs_list),columns=cols)
    
    return estimates_df

In [41]:
itr_nbr = 100
dept_lambda_values={
    7:[0.0],
    11:[0.0],
    14:[0.0]
}

In [42]:
%%time
# code for running for all dept
for dept in dept_lambda_values.keys():
    print('dept :',dept)

    #create modelling dataset
    df = createModellingDataset(item_matl, dept, 'fc_lb', 1)
    
    print(df.shape)
    pd.Series(df.index).to_csv(bootstrap_estimates_path 
                               + '/mds_fam_id_train_dept_' + str(dept) + '_iter_' + str(itr_nbr) \
                               + '.csv',index=False)
    
    #create upper and lower bounds for each materials
    misc_cost_groups = return_misc_cost_group_per_dept(item_matl,dept)
    
    lb,ub = lowerAndUpperBounds(df,'fc_lb',
                              ihs_coo_estimates_ul,ihs_coo_estimates_ll,
                              ihs_estimates_ul,ihs_estimates_ll,
                              single_item_coo_estimates,single_item_estimates,misc_cost_groups,
                              np.inf,0.01)



    for loss_lambda in dept_lambda_values[dept]:
        
        print('loss_lambda :',loss_lambda)
        
        estimates_df = runBootStrapGDModel(df
                                     , misc_cost_groups
                                     , lb
                                     , ub
                                     , epochs=10**6
                                     , verbose=True
                                     , method='ADAM'
                                     , noIterations=10
                                     , runBootStrap=True
                                     , loss_lambda = loss_lambda)
    
        estimates_df.to_pickle(bootstrap_estimates_path
                           +'/estimates_dept_' + str(dept)+ '_lambda_value_' + str(loss_lambda) 
                            + '_iter_' + str(itr_nbr) +'.pkl')

dept : 7
(1789, 267)
loss_lambda : 0.0
step: 0, obj = 0.12
step: 760, obj = 0.01
********** bootstrap regression started **********
***** bootstrap iteration 0 *****
step: 0, obj = 0.04
step: 399, obj = 0.01
********** bootstrap regression done **********
dept : 11
(912, 192)
loss_lambda : 0.0
step: 0, obj = 0.11
step: 1248, obj = 0.01
********** bootstrap regression started **********
***** bootstrap iteration 0 *****
step: 0, obj = 0.04
step: 503, obj = 0.01
********** bootstrap regression done **********
dept : 14
(1931, 299)
loss_lambda : 0.0
step: 0, obj = 0.78
step: 1526, obj = 0.01
********** bootstrap regression started **********
***** bootstrap iteration 0 *****
step: 0, obj = 0.42
step: 1602, obj = 0.01
********** bootstrap regression done **********
CPU times: user 4min 23s, sys: 1min 16s, total: 5min 40s
Wall time: 3min 4s
